# Object Detection Evaluation 

Print the relative error on object detection for VAE or GAN approach.

In [ ]:
import warnings
warnings.simplefilter(action="ignore", category=FutureWarning)

import seaborn as sns
from matplotlib import pyplot as plt
import pandas as pd
import os

In [ ]:
sns.set()
sns.set_style("white")

In [ ]:
import matplotlib
%matplotlib inline

import matplotlib.patches as mpatches

matplotlib.use("pgf")
matplotlib.rcParams.update({
    "pgf.texsystem": "pdflatex",
    "font.family": "serif",
    "text.usetex": True,
    "pgf.rcfonts": False,
})

matplotlib.rcParams.update({'font.size': 9})

In [ ]:
df = pd.DataFrame()

csv_files_folder = "./object_detection_files_KITTI_NEW"
for csv_file in os.listdir(csv_files_folder):
    df_temp = pd.read_csv(os.path.join(csv_files_folder, csv_file))
    df = df.append(df_temp)

df.loc[df["model"]=="GAN", "image_name"].str.split("_").str[0] + ".png"
min_confidence_score = 0.7
df = df[df["score"] >= min_confidence_score]
df = df.loc[df["class"].isin([1.0, 2.0, 3.0, 4.0])] #class
df.loc[df["model"]=="GAN", "image_name"] = df.loc[df["model"]=="GAN", "image_name"].str.split("_").str[0] + ".png"

df.head()

In [ ]:
df_vae = df[df["model"] == "VAE"]
df_gan = df[df["model"] == "GAN"]
df_orig = df[df["model"] == "ORIG"]

df_vae_counts = df_vae.groupby(["image_name", "quality_level", "model"]).size().reset_index(name="counts").dropna()
df_vae_counts = df_vae_counts.pivot(index="image_name", columns=["quality_level", "model"], values="counts").dropna()
df_gan_counts = df_gan.groupby(["image_name", "quality_level", "model"]).size().reset_index(name="counts").dropna()
df_gan_counts = df_gan_counts.pivot(index="image_name", columns=["quality_level", "model"], values="counts").dropna()
df_orig_counts = df_orig.groupby(["image_name", "quality_level", "model"]).size().reset_index(name="counts").dropna()
df_orig_counts = df_orig_counts.pivot(index="image_name", columns=["quality_level", "model"], values="counts").dropna()

In [ ]:
print(df_vae["quality_level"].unique())
print(df_gan["quality_level"].unique())

In [ ]:
df_diff_vae = pd.DataFrame()
df_diff_gan = pd.DataFrame()

for quality_level in df_vae["quality_level"].unique():
    df_diff_vae[quality_level] = (df_vae_counts[quality_level]["VAE"] - df_orig_counts["original"]["ORIG"]) / df_orig_counts["original"]["ORIG"]

for quality_level in df_gan["quality_level"].unique():
    df_diff_gan[quality_level] = (df_gan_counts[quality_level]["GAN"] - df_orig_counts["original"]["ORIG"]) / df_orig_counts["original"]["ORIG"]

df_diff_vae = df_diff_vae.reset_index()
df_diff_vae = pd.melt(df_diff_vae,  id_vars=["image_name"], value_vars=df_vae["quality_level"].unique())
df_diff_vae["model"] = "VAE"

df_diff_gan = df_diff_gan.reset_index()
df_diff_gan = pd.melt(df_diff_gan,  id_vars=["image_name"], value_vars=df_gan["quality_level"].unique())
df_diff_gan["model"] = "GAN"

df_diff = df_diff_gan.append(df_diff_vae)
df_diff["value"] = df_diff["value"] * 100
df_diff = df_diff.reset_index()
df_diff.head()

In [ ]:
df_diff["variable"].unique()

In [ ]:
df_diff_avg = df_diff.groupby(["variable", "model"]).agg(["mean","std"])
df_diff_avg

In [ ]:
df_diff["variable"].unique()

In [ ]:
df_eval = pd.DataFrame()

csv_files_folder = "./evaluation_files_KITTI"
for csv_file in os.listdir(csv_files_folder):
    df_temp = pd.read_csv(os.path.join(csv_files_folder, csv_file))
    df_eval = df_eval.append(df_temp, ignore_index=True)
df_eval.loc[df_eval["model"]=="GAN", "input_filename"] = df_eval.loc[df_eval["model"]=="GAN", "input_filename"].astype(str).str.zfill(6).str.split(".").str[0] + ".png"

df_eval = df_eval[((df_eval["model"] == "VAE") & (df_eval["input_shape"] == "(256, 256)")) | (df_eval["model"] == "GAN")]
df_eval = df_eval.drop(["input_shape", "output_filename"], axis=1)
df_eval

In [ ]:
df_diff = df_diff[["image_name", "variable", "value", "model"]].set_axis(["input_filename", "quality_level", "value", "model"], axis="columns")
df_diff

In [ ]:
dff = pd.merge(df_eval, df_diff, how="inner", on=["input_filename", "quality_level", "model"])[["input_filename", "model", "quality_level", "value", "q_bpp"]].dropna()
dff

In [ ]:
print(dff[dff["model"] == "GAN"].count()["model"])
print(dff[dff["model"] == "VAE"].count()["model"])

In [ ]:
fig = plt.figure(figsize=(4,2), tight_layout=True)
data = dff[dff["q_bpp"] <= 1.0]

plt.xlabel("Difference [%]")
plt.ylabel("Bit-rate [bpp]")
fig = sns.kdeplot(data=data[data["model"] == "VAE" ], x="value", y="q_bpp", levels=30, shade=True, alpha=.5, cut=20)
fig = sns.kdeplot(data=data[data["model"] == "GAN" ], x="value", y="q_bpp", levels=30, shade=True, alpha=.5, cut=20)
plt.xlim(-125, 125)
plt.ylim(0, 1)
vae_kde = mpatches.Patch(color="#1f77b4", label='VAE')
gan_kde = mpatches.Patch(color="#ff7f0e", label='GAN')
plt.legend(handles=[vae_kde, gan_kde], fontsize="x-small")

plt.savefig(f"for_latex/object_detection_kde.pgf", dpi=400)
plt.savefig(f"for_latex/object_detection_kde.png", dpi=400)

plt.show()